In [1]:
import pandas as pd

In [2]:
# Read the GSM CSV file into a DataFrame
df = pd.read_csv('GSM.csv')

In [3]:
# Convert 'BCCH' column to numeric type
df['BCCH'] = pd.to_numeric(df['BCCH'], errors='coerce')

In [4]:
def check_co_channel_interference(df):
    # Create a list to store results
    results = []

    # Group the DataFrame by 'SiteID'
    grouped = df.groupby(['SiteID', 'SiteName'])
    
    # Iterate over each group
    for (site, site_name), data in grouped:
        # Initialize a set to store unique BCCH values for each CellID
        bcch_dict = {}
        
        # Iterate over each row in the group and store BCCH values in the dictionary
        for _, row in data.iterrows():
            cell_id = row['CellID']
            bcch = row['BCCH']
            bcch_dict[cell_id] = bcch
        
        # Check for co-channel interference within each CellID
        for cell1, bcch1 in bcch_dict.items():
            for cell2, bcch2 in bcch_dict.items():
                if cell1 != cell2 and bcch1 == bcch2:
                    results.append((site, site_name, cell1, cell2, 'Co-Channel Interference'))
    
    # Create a DataFrame from the results list
    co_channel_df = pd.DataFrame(results, columns=['SiteID', 'SiteName', 'CellID1', 'CellID2', 'Interference'])

    return co_channel_df


In [5]:
def check_adjacent_channel_interference(df):
    # Create a list to store results
    results = []

    # Group the DataFrame by 'SiteID'
    grouped = df.groupby(['SiteID', 'SiteName'])
    
    # Iterate over each group
    for (site, site_name), data in grouped:
        # Initialize a dictionary to store BCCH values for each CellID
        bcch_dict = {}
        
        # Iterate over each row in the group and store BCCH values in the dictionary
        for _, row in data.iterrows():
            cell_id = row['CellID']
            bcch = row['BCCH']
            bcch_dict[cell_id] = bcch
        
        # Check for adjacent channel interference within each CellID
        for cell_id, bcch in bcch_dict.items():
            # Check the BCCH difference with the adjacent CellIDs
            adjacent_bcch_1 = bcch_dict.get(cell_id - 1)
            adjacent_bcch_2 = bcch_dict.get(cell_id + 1)
            if adjacent_bcch_1 is not None and abs(bcch - adjacent_bcch_1) == 1:
                results.append((site, site_name, cell_id, cell_id - 1, 'Adjacent Channel Interference'))
            elif adjacent_bcch_2 is not None and abs(bcch - adjacent_bcch_2) == 1:
                results.append((site, site_name, cell_id, cell_id + 1, 'Adjacent Channel Interference'))
    
    # Create a DataFrame from the results list
    adj_channel_df = pd.DataFrame(results, columns=['SiteID', 'SiteName', 'Source_CellID', 'Target_CellID', 'Interference'])

    return adj_channel_df


In [6]:

# Call the functions to get co-channel and adjacent channel interference DataFrames
co_channel_df = check_co_channel_interference(df)
adj_channel_df = check_adjacent_channel_interference(df)


In [7]:
# Create an ExcelWriter object
with pd.ExcelWriter('GSM_interference_results_Batch7.xlsx') as writer:
    # Write co-channel DataFrame to a sheet
    co_channel_df.to_excel(writer, sheet_name='Co-Channel Interference', index=False)
    
    # Write adjacent channel DataFrame to a sheet
    adj_channel_df.to_excel(writer, sheet_name='Adjacent Channel Interference', index=False)
